In [ ]:
import google.generativeai as genai
import pyaudio
from gtts import gTTS
import speech_recognition as sr
import os
import re

In [ ]:
# Activate a few functions

def speak_question(question):
    tts = gTTS(question)
    tts.save("question.mp3")
    os.system("afplay question.mp3")  # For MacOS, use "start question.mp3" on Windows      


def listen_to_applicant(timeout=None, phrase_time_limit=30, ambient_duration=1.0, pause_threshold=1.8, non_speaking_duration=0.8, energy_threshold=None, show_prompts=True):
    """
    Listens to the applicant and converts to text.

    Parameters:
    - timeout: max seconds to wait for the phrase to start (None = wait indefinitely)
    - phrase_time_limit: max seconds for a single phrase (None = no limit). Increase to allow long answers.
    - ambient_duration: seconds to adjust for ambient noise before listening
    - pause_threshold: seconds of silence that indicate end of phrase (increase to wait longer)
    - non_speaking_duration: how long to keep non-speaking buffer (increase to be more tolerant)
    - energy_threshold: if set, fixes energy threshold (disable dynamic adjustment)
    - show_prompts: print status messages
    """
    r = sr.Recognizer()
    r.pause_threshold = pause_threshold
    r.non_speaking_duration = non_speaking_duration

    if energy_threshold is not None:
        r.energy_threshold = energy_threshold
        r.dynamic_energy_threshold = False
    else:
        r.dynamic_energy_threshold = True

    with sr.Microphone() as source:
        if show_prompts:
            print("Adjusting for ambient noise... (keep quiet)")
        r.adjust_for_ambient_noise(source, duration=ambient_duration)
        if show_prompts:
            print("Listening... (speak; pause to finish)")

        try:
            audio = r.listen(source, timeout=timeout, phrase_time_limit=phrase_time_limit)
        except sr.WaitTimeoutError:
            if show_prompts:
                print("Listening timed out waiting for phrase to start.")
            return ""
        except Exception as e:
            if show_prompts:
                print(f"Error while listening: {e}")
            return ""

    try:
        text = r.recognize_google(audio)
        if show_prompts:
            print(f"Applicant said: {text}")
        return text
    except sr.UnknownValueError:
        if show_prompts:
            print("Could not understand audio")
        return ""
    except sr.RequestError as e:
        if show_prompts:
            print(f"Speech recognition service error: {e}")
        return ""


In [ ]:
# Initialize the Gemini API

from dotenv import load_dotenv
import os
load_dotenv()

try:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    genai.configure(api_key=GOOGLE_API_KEY)
    gemini_model = genai.GenerativeModel('gemini-flash-latest') 

    # Re-create the fictitious job posting and CV files in the /content directory
    job_posting_content = """
    Title: Machine Learning Engineer
    Company: Google
    Location: Mountain View, CA

    About the Role:
    Google is seeking a talented Machine Learning Engineer to join our innovative team. You will be responsible for designing, developing, and deploying scalable ML models that power various Google products. This role requires strong programming skills, a solid understanding of machine learning principles, and experience with large-scale data processing.

    Responsibilities:
    - Design and implement machine learning models for specific applications.
    - Develop and maintain robust data pipelines.
    - Collaborate with cross-functional teams to integrate ML solutions.
    - Evaluate and improve the performance of ML models.
    - Stay up-to-date with the latest advancements in machine learning.

    Qualifications:
    - Bachelor's or Master's degree in Computer Science, related field, or equivalent practical experience.
    - 3+ years of experience in machine learning or related field.
    - Proficiency in Python and ML frameworks (e.g., TensorFlow, PyTorch).
    - Experience with cloud platforms (e.g., Google Cloud, AWS).
    - Strong problem-solving and communication skills.

    Bonus Qualifications:
    - Experience with natural language processing or computer vision.
    - Publications in top-tier ML conferences.
    """

    applicant_cv_content = """
    Name: Jane Doe
    Email: jane.doe@email.com
    Phone: 555-123-4567
    LinkedIn: linkedin.com/in/janedoe

    Summary:
    Highly motivated and results-oriented Machine Learning Engineer with 4 years of experience in developing and deploying ML solutions. Proven ability to work with large datasets and implement complex models. Seeking a challenging role at a leading tech company like Google.

    Experience:
    Senior ML Engineer, Tech Innovations Inc., 2020 - Present
    - Led the development of a recommendation system using TensorFlow, resulting in a 15% increase in user engagement.
    - Built and maintained data pipelines for processing large-scale datasets.
    - Collaborated with product teams to integrate ML models into production systems.

    ML Engineer, Data Solutions Ltd., 2018 - 2020
    - Developed and deployed machine learning models for fraud detection.
    - Worked with cloud platforms (AWS) to manage and scale ML workloads.
    - Participated in code reviews and mentored junior engineers.

    Education:
    Master of Science in Computer Science, Stanford University, 2018
    Bachelor of Science in Computer Science, University of California, Berkeley, 2016

    Skills:
    Programming Languages: Python, Java, C++
    ML Frameworks: TensorFlow, PyTorch, scikit-learn
    Cloud Platforms: Google Cloud, AWS
    Other: SQL, Docker, Kubernetes
    """

    with open('./content/job_posting.txt', 'w') as f:
        f.write(job_posting_content)

    with open('./content/applicant_cv.txt', 'w') as f:
        f.write(applicant_cv_content)

    print("Re-created '/content/job_posting.txt' and '/content/applicant_cv.txt' with sample content.")

    # Now, attempt to load the files and perform the analysis using Gemini
    import os
    import re

    job_posting_path = './content/job_posting.txt'
    applicant_cv_path = './content/applicant_cv.txt'

    if os.path.exists(job_posting_path) and os.path.exists(applicant_cv_path):
        try:
            with open(job_posting_path, 'r') as f:
                job_posting_text = f.read()

            with open(applicant_cv_path, 'r') as f:
                applicant_cv_text = f.read()

            # Re-define company_research_results
            company_name_match = re.search(r"Company: (.+)", job_posting_text)
            if company_name_match:
                company_name = company_name_match.group(1).strip()
                company_research_results = f"Simulated research results for {company_name}: Company is a major tech company known for innovation in AI and search. Culture is fast-paced and collaborative. Additional responsibilities for ML Engineers often involve cross-team collaboration and contributing to open-source projects."
            else:
                company_name = "the company"
                company_research_results = "Simulated research results: General information about tech companies."


            print("Performing analysis and comparison of job posting, CV, and company research using Gemini...")

            # Construct a prompt for the gemini_model
            prompt = f"""You are an AI interview assistant. Analyze the following job posting, applicant CV, and company research results to identify:
            1. Matching skills between the job posting and the CV.
            2. Relevant experience from the CV that aligns with the job posting.
            3. Potential cultural fit notes based on the company research and CV.
            4. Identified gaps in skills or experience based on the job posting and CV, considering bonus qualifications and company research.

            Provide the results in a structured format with clear headings for each category:

            Job Posting:
            {job_posting_text}

            Applicant CV:
            {applicant_cv_text}

            Company Research Results:
            {company_research_results}

            Analysis Results:
            """

            # Call the gemini_model to generate the analysis results
            # Use a try-except block to handle potential errors during model inference
            try:
                response = gemini_model.generate_content(prompt)
                analysis_text = response.text
                print("\nRaw Analysis Results from Gemini:")
                print(analysis_text)

                # Parse the model's response text to extract the analysis results
                # This is a basic parsing based on expected headings. More robust parsing might be needed.
                analysis_results = {}
                current_category = None
                for line in analysis_text.split('\n'):
                    line = line.strip()
                    if line.startswith("Matching skills:"):
                        current_category = "matching_skills"
                        analysis_results[current_category] = []
                    elif line.startswith("Relevant experience:"):
                        current_category = "relevant_experience"
                        analysis_results[current_category] = []
                    elif line.startswith("Potential cultural fit notes:"):
                        current_category = "cultural_fit_notes"
                        analysis_results[current_category] = ""
                    elif line.startswith("Identified gaps:"):
                        current_category = "identified_gaps"
                        analysis_results[current_category] = []
                    elif current_category:
                        if current_category in ["matching_skills", "relevant_experience", "identified_gaps"]:
                            if line.startswith("- "):
                                analysis_results[current_category].append(line[2:].strip())
                        elif current_category == "cultural_fit_notes":
                            analysis_results[current_category] += line.strip() + " "

                print("\nParsed Analysis Results:")
                for key, value in analysis_results.items():
                    print(f"{key}: {value}")

            except Exception as e:
                print(f"An error occurred during Gemini analysis: {e}")
                # Define a placeholder in case of error to allow subsequent cells to run
                analysis_results = {
                    "matching_skills": ["Analysis failed"],
                    "relevant_experience": ["Analysis failed"],
                    "cultural_fit_notes": "Analysis failed",
                    "identified_gaps": ["Analysis failed"]
                }
                print("\nUsing placeholder analysis_results due to error.")


        except FileNotFoundError:
            print("Error: File not found after re-creation. This is unexpected.")
            job_posting_text = None
            applicant_cv_text = None
            company_research_results = None
            analysis_results = None # Ensure analysis_results is None if file loading fails
        except Exception as e:
            print(f"An error occurred while reading the files or performing initial steps: {e}")
            job_posting_text = None
            applicant_cv_text = None
            company_research_results = None
            analysis_results = None # Ensure analysis_results is None if error occurs
    else:
        print("Error: Files do not exist in the specified directory after re-creation. This is unexpected.")
        job_posting_text = None
        applicant_cv_text = None
        company_research_results = None
        analysis_results = None # Ensure analysis_results is None if files don't exist

except Exception as e:
    print(f"An error occurred during Gemini model initialization: {e}")
    job_posting_text = None
    applicant_cv_text = None
    company_research_results = None
    analysis_results = None # Ensure analysis_results is None if model initialization fails


In [ ]:
# Assuming analysis_results and gemini_model are available from previous steps.

print("Generating interview questions using Gemini based on analysis results...")

# Define a prompt for the gemini_model
prompt = f"""You are an AI interview assistant. Based on the following analysis of a job posting, applicant CV, and company research, generate a list of tailored interview questions.
Focus on creating questions that probe the applicant's strengths in matching areas and explore the identified gaps. Also, include questions related to cultural fit based on the company research.
Provide only the list of questions, one per line, starting with a number.

Analysis Results:
Matching skills: {analysis_results.get('matching_skills', 'N/A')}
Relevant experience: {analysis_results.get('relevant_experience', 'N/A')}
Cultural fit notes: {analysis_results.get('cultural_fit_notes', 'N/A')}
Identified gaps: {analysis_results.get('identified_gaps', 'N/A')}

Generate Interview Questions:
"""

simulated_interview_questions = []

# Call the gemini_model to generate the interview questions
# Use a try-except block to handle potential errors during model inference
try:
    response = gemini_model.generate_content(prompt)
    questions_text = response.text
    print("\nRaw Generated Questions from Gemini:")
    print(questions_text)

    # Parse the model's response text to extract the interview questions
    # Assuming each question is on a new line and starts with a number
    for line in questions_text.split('\n'):
        line = line.strip()
        if line and (line[0].isdigit() or line.startswith('-')): # Basic check for numbered or bulleted list
             simulated_interview_questions.append(line.split('. ', 1)[-1].strip() if line[0].isdigit() else line[2:].strip())


    if not simulated_interview_questions:
         print("\nWarning: No questions were parsed from the model's response. Using default questions.")
         # Assign a default list of questions if parsing failed
         simulated_interview_questions = [
            "Tell me about yourself.",
            "Why are you interested in this role?",
            "What are your strengths and weaknesses?",
            "Do you have any questions for me?"
        ]


except Exception as e:
    print(f"An error occurred during Gemini question generation: {e}")
    # Assign a default list of questions in case of error
    simulated_interview_questions = [
        "Tell me about yourself.",
        "Why are you interested in this role?",
        "What are your strengths and weaknesses?",
        "Do you have any questions for me?"
    ]
    print("\nUsing default interview questions due to error.")


print("\nSimulated Interview Questions:")
for i, question in enumerate(simulated_interview_questions):
    print(f"{i+1}. {question}")

In [ ]:
# Assuming simulated_interview_questions, speak_question, listen_to_applicant,
# simulated_score, and hiring_decision variables are defined in previous cells.

print("Starting the simulated interview...")

# AI Introduction
intro_message = "Hello, I am your AI interviewer. I will be asking you a series of questions about your background and experience to assess your suitability for the role. Please respond verbally after each question. The interview will take approximately 15 minutes. Do you have any questions before we begin?"
speak_question(intro_message)

# In a real application, you might listen for a response here to the intro question.
# For this simulation, we'll proceed directly to the interview questions.
applicant_intro_response = listen_to_applicant() # Uncomment in a real scenario

interview_responses = {}

for i, question in enumerate(simulated_interview_questions):
    print(f"\nAsking question {i+1}: {question}")
    speak_question(question)

    print("Waiting for applicant response...")
    # In a real application, you would uncomment the line below:
    applicant_response = listen_to_applicant() # Uncomment this in a real interactive scenario
    # For simulation purposes, you might use a placeholder if not running interactively
    # applicant_response = f"Simulated response to question {i+1}"

    interview_responses[question] = applicant_response
    print(f"Applicant response recorded: {applicant_response}")

print("\nSimulated interview concluded.")
print("\nInterview Responses:")
for question, response in interview_responses.items():
    print(f"Q: {question}")
    print(f"A: {response}")
    print("-" * 20)

# Simulated applicant thanking the interviewer
print("\nApplicant thanking the interviewer (simulated)...")
# In a real application, this would be a response captured by listen_to_applicant()
simulated_thank_you = "Thank you for your time today. I enjoyed learning more about this opportunity."
# If you had the interviewer's name, you could add it here, e.g., "Thank you, [Interviewer Name], for your time today."
print(f"Simulated applicant says: {simulated_thank_you}")




In [ ]:
# Initialize the Gemini API - necessary for this step
# import google.generativeai as genai
# from google.colab import userdata

try:
    # GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    # genai.configure(api_key=GOOGLE_API_KEY)
    # gemini_model = genai.GenerativeModel('gemini-pro') # Or another suitable model

    print("Gemini model initialized successfully for scoring.")

    # Define placeholder simulated_interview_questions and interview_responses if they are not already defined
    if 'simulated_interview_questions' not in locals() or not simulated_interview_questions:
        simulated_interview_questions = [
            "Your CV highlights experience with TensorFlow and PyTorch. Can you describe a project where you used one of these frameworks to build a scalable ML model?",
            "You've built and maintained data pipelines. What are some of the key challenges you've faced in this area and how did you address them?",
            "The job posting mentions collaborating with cross-functional teams. Can you give an example of a time you successfully collaborated with a non-ML team?",
            "Our company culture values collaboration and contributions to the wider tech community. How do you approach collaboration, and have you contributed to any open-source projects?",
            "The job posting listed natural language processing and computer vision as bonus qualifications. Do you have any experience or interest in these areas?",
            "Publications in top-tier ML conferences were also listed as a bonus. Have you published any research, or are you interested in pursuing publications?",
            "Based on your understanding of Google's work in AI, what kind of ML projects are you most excited about working on?",
            "Do you have any questions for me about the role or the company?"
        ]
        print("Using placeholder simulated_interview_questions.")

    if 'interview_responses' not in locals() or not interview_responses:
        # Create simulated responses for the placeholder questions
        interview_responses = {
            simulated_interview_questions[0]: "Simulated response to question 1: I worked on a recommendation system using TensorFlow that scaled to millions of users.",
            simulated_interview_questions[1]: "Simulated response to question 2: Challenges included data inconsistency, which I addressed by implementing robust validation checks.",
            simulated_interview_questions[2]: "Simulated response to question 3: I collaborated with the marketing team to integrate an ML model for targeted advertising.",
            simulated_interview_questions[3]: "Simulated response to question 4: I believe in open communication and have contributed to a small open-source data processing library.",
            simulated_interview_questions[4]: "Simulated response to question 5: I have some academic experience in NLP and am very interested in learning more about CV.",
            simulated_interview_questions[5]: "Simulated response to question 6: I haven't published yet, but it's a goal I'm actively working towards.",
            simulated_interview_questions[6]: "Simulated response to question 7: I'm most excited about Google's work in responsible AI and applying ML to solve real-world problems.",
            simulated_interview_questions[7]: "Simulated response to question 8: Yes, can you describe the typical day-to-day of an ML Engineer on your team?"
        }
        print("Using placeholder interview_responses.")


    print("\nProcessing interview responses and calculating score using Gemini...")

    # Construct a detailed prompt for Gemini to analyze the interview responses
    scoring_prompt = f"""You are an AI interview assistant. Evaluate the following interview responses based on the provided job posting and interview questions.
    Assess the applicant's responses for:
    - Relevance to the questions and job requirements.
    - Technical accuracy and depth (where applicable).
    - Communication skills (clarity, conciseness).
    - Alignment with the company culture and values (based on the simulated company research provided earlier in the process).

    Provide a score out of 100 and a hiring recommendation.

    Job Posting:
    {job_posting_text if 'job_posting_text' in locals() and job_posting_text else 'N/A'}

    Interview Questions and Responses:
    """
    for question, response in interview_responses.items():
        scoring_prompt += f"Q: {question}\nA: {response}\n---\n"

    scoring_prompt += """

    Based on this evaluation, provide a score (out of 100) and a hiring recommendation. Format your response clearly, like this:

    Score: [Score out of 100]
    Recommendation: [Hiring Recommendation]
    """

    # Call the gemini_model to generate the score and recommendation
    gemini_score = None
    gemini_hiring_decision = "Could not get recommendation from AI."

  
    try:
        response = gemini_model.generate_content(scoring_prompt)
        scoring_text = response.text
        print("\nRaw Scoring Results from Gemini:")
        print(scoring_text)

        # Parse the model's response to extract the score and recommendation
        score_match = re.search(r"Score: (\d+)", scoring_text)
        if score_match:
            gemini_score = int(score_match.group(1))

        recommendation_match = re.search(r"Recommendation: (.+)", scoring_text)
        if recommendation_match:
            gemini_hiring_decision = recommendation_match.group(1).strip()
        else:
            # Fallback: if we have a score but no explicit recommendation, derive one from the score
            if gemini_score is not None:
                hiring_threshold = 70
                gemini_hiring_decision = ("Candidate meets criteria." 
                                          if gemini_score >= hiring_threshold 
                                          else "Candidate does not meet criteria.")
            # otherwise keep the default gemini_hiring_decision
    except Exception as e:
        print(f"An error occurred during Gemini scoring: {e}")
        print("Using fallback scoring due to AI error.")


    # Use Gemini's results if available, otherwise use the basic placeholder logic
    if gemini_score is not None:
        simulated_score = gemini_score
        hiring_decision = gemini_hiring_decision
        print("\nUsing Gemini's score and recommendation.")
    else:
        # Fallback to basic placeholder scoring if Gemini call failed
        num_questions = len(simulated_interview_questions)
        num_responses = len(interview_responses)
        simulated_score = int((num_responses / num_questions) * 100) if num_questions > 0 else 0
        hiring_decision = "Could not get AI recommendation. Fallback decision: "
        hiring_threshold = 70 # Example threshold
        if simulated_score >= hiring_threshold:
             hiring_decision += "Candidate meets basic criteria."
        else:
             hiring_decision += "Candidate does not meet basic criteria."
        print("\nUsing fallback score and recommendation.")


    print(f"\nApplicant's simulated interview score: {simulated_score}")
    print(f"Hiring Decision: {hiring_decision}")
    conclusion_message = f"Thank you for completing the interview. The interview has now concluded. Your simulated score is {simulated_score}. The hiring recommendation is: {hiring_decision}."
    speak_question(conclusion_message)
except Exception as e:
    print(f"An error occurred during Gemini model initialization for scoring: {e}")
    print("\nSkipping scoring and decision due to model initialization error.")
    # Provide a placeholder score and decision if model initialization fails
    simulated_score = "N/A"
    hiring_decision = "Could not perform scoring due to AI model error."
    print(f"\nApplicant's simulated interview score: {simulated_score}")
    print(f"Hiring Decision: {hiring_decision}")